# LLM-based Final Scoring

If resumes make it to this stage, we can assume they meet the minimum requirements. Here, our job is to filter out ~80% of resumes

At the end of this, let's compare our scoring methodology to a baseline: feeding the entire resume and asking the LLM to make a recommendation

In [13]:
import json
from pprint import pprint
from pydantic import BaseModel, Field

from utils.with_structured_output import with_structured_output

In [3]:
with open("../output/parsed_resume_info.json", "r") as file:
    parsed_resume_info = json.load(file)
    
with open("../output/parsed_job_desc.json", "r") as file:
    parsed_job_info = json.load(file)

In [16]:
class ResumeScore(BaseModel):
    score: int                  = Field(..., alias="Score", ge=0, le=10)
    justification: list[str]    = Field(..., alias="Justification")

In [17]:
FINAL_SCORING_TEMPLATE = """
You are an expert resume evaluator. Given a parsed resume and parsed job description, both represented as JSON strings, your job is to evaluate how well the resume fits the job description.

Please output a score between 0 and 10 that measures the degree of fit, where:
    0 – 2: Not recommended
        Candidate has significant shortcomings that outweigh any strengths.
    3 – 4: Weak fit
        Candidate meets some qualifications but lacks essential skills or experiences necessary for the role.
    5 – 6: Average fit
        Candidate meets minimum qualifications and shows potential but may required **significant** training or support.
    7 – 8: Strong fit
        Candidate is well-qualified and likely to succeed with minimal training or onboarding. **A strong contender for the role.**
    9 – 10: Exceptional fit
        Candidate exceeds expectations and demonstrates outstanding qualifications, skills, or cultural alignment. **A top-tier hire.**
        
In addition to your score, please output comprehensive feedback as a list of concise points that justifies your score. 

Output format:
    {{
        "Score": <Integer between 0-10 that describes resume's alignment>,
        "Justification": ["list", "of", "points", "justifying", "score"]
    }}
---
Resume:
{resume_info}

Job Description:
{job_info}

Output:
"""

In [18]:
final_score = with_structured_output(
    prompt=FINAL_SCORING_TEMPLATE.format(
        resume_info=parsed_resume_info,
        job_info=parsed_job_info),
    schema=ResumeScore)

In [ ]:
pprint(final_score)

### OpenAI o1-mini Reasoning Model